In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir  ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d slothkong/10-monkey-species

 98% 537M/547M [00:06<00:00, 56.6MB/s]
100% 547M/547M [00:06<00:00, 92.0MB/s]


In [ ]:
from zipfile import ZipFile
file_name = '10-monkey-species.zip'

with ZipFile(file_name,'r') as zi:
  zi.extractall()
  print('s')

s


In [ ]:
from keras.applications import MobileNet

#Loading imagenet weights
img_rows,img_col = 224,224

MobileNet= MobileNet(weights = 'imagenet',include_top=False,input_shape=(img_rows,img_col,3))

for layer in MobileNet.layers:
    layer.trainable = False

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
for i, layer in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [ ]:
#from keras.models import Sequential
def addmodel(model,num_classes):
    
    top_model=model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation = 'relu')(top_model)
    top_model = Dense(512,activation = 'relu')(top_model)
    top_model =Dense(num_classes,activation = 'softmax')(top_model)
    return top_model
    

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,Flatten
from keras.layers import GlobalAveragePooling2D,ZeroPadding2D
from keras.models import Model
from keras.layers.normalization import BatchNormalization

num_classes=10

FC_head = addmodel(MobileNet,num_classes)

model = Model(inputs = MobileNet.input, outputs=FC_head)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

training_dir = '/content/training/training'
test_dir = '/content/validation/validation'

train_data= ImageDataGenerator(
    rescale = 1./255,
    rotation_range =45,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode = 'nearest')

test_data = ImageDataGenerator(rescale = 1./255)

batch_size = 32

train_gen = train_data.flow_from_directory(
    training_dir,
    batch_size = batch_size,
    target_size = (img_rows,img_col),
    class_mode = 'categorical')

test_gen = test_data.flow_from_directory(
    test_dir,
    batch_size = batch_size,
    target_size = (img_rows,img_col),
    class_mode = 'categorical')
    
    

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [ ]:

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint = ModelCheckpoint("/content/mokey_breed_mobilenet.h5",
                            monitor='val_loss',
                            mode = 'min',
                            save_best_only= True,
                            verbose =1)

stopping = EarlyStopping(monitor ='val_loss',
                        min_delta = 0,
                       patience = 3,
                        restore_best_weights=True)
callbacks =[checkpoint,stopping]

model.compile(loss ='categorical_crossentropy',
             optimizer = Adam(),
              metrics = ['accuracy'])
nb_train_samples = 1097
nb_validation_samples = 272

history = model.fit_generator(train_gen,
                             steps_per_epoch = nb_train_samples//32,
                             epochs =  10,
                             callbacks = callbacks,
                             validation_data = test_gen,
                              validation_steps = nb_validation_samples // 32)


Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 0.3876 - accuracy: 0.8771
Epoch 00001: val_loss improved from inf to 0.17773, saving model to /content/mokey_breed_mobilenet.h5
34/34 [==============================] - 87s 3s/step - loss: 0.3876 - accuracy: 0.8771 - val_loss: 0.1777 - val_accuracy: 0.9453
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 0.1751 - accuracy: 0.9437
Epoch 00002: val_loss improved from 0.17773 to 0.16610, saving model to /content/mokey_breed_mobilenet.h5
34/34 [==============================] - 87s 3s/step - loss: 0.1751 - accuracy: 0.9437 - val_loss: 0.1661 - val_accuracy: 0.9531
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 0.2144 - accuracy: 0.9231
Epoch 00003: val_loss did not improve from 0.16610
34/34 [==============================] - 86s 3s/step - loss: 0.2144 - accuracy: 0.9231 - val_loss: 0.2035 - val_accuracy: 0.9531
Epoch 4/10
34/34 [==============================] - ETA: 0s - loss: 0.15

In [ ]:
from keras.models import load_model

classifier = load_model('/content/mokey_breed_mobilenet.h5')

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from google.colab.patches import cv2_imshow

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2_imshow(expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage('/content/validation/validation/')
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.